In [30]:
import numpy as np
import tensorflow as tf
from keras import backend as K
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def read_csv_data(file_name, debug, server=False, num_rows=200):
    if server:
        path = '/home/science/data/'
        df = pd.read_hdf(path + file_name + '.h5', 'data')
    else:
        path = '/home/gublu/Desktop/THINKSTATS/Competition/data/'
        if debug:
            df = pd.read_csv(path + file_name + '.csv', nrows=num_rows)
        else:
            df = pd.read_csv(path + file_name + '.csv')
    for col in list(df):
        if str(df[col].dtype) == 'category':
            df[col] = df[col].astype('object')
    return df
def label_encode_it(df):
    encode_these_columns = []
    for col in list(df):
        if col == 'TARGET':
            continue
        if str(df[col].dtype) in ['object', 'category']:
            encode_these_columns.append(col)
            df[col] = df[col].astype('category').cat.codes
    print(encode_these_columns, '**********')
    return df

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [2]:
# load dataset
train_df = read_csv_data('shiv_train', debug=True)


In [3]:
train_df.head()

,SK_ID_CURR,TARGET,DAYS_BIRTH,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,NEW_IS_DAYS_EMPLOYED_365243,DAYS_EMPLOYED,AMT_CREDIT,AMT_ANNUITY,...,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MIN,POS_MONTHS_BALANCE_VAR,POS_SK_ID_PREV_NUNIQUE,POS_SK_DPD_MEAN,POS_CNT_INSTALMENT_FIRST,POS_DPD_MINUS_DPD_DEF_MAX,POS_LATEST_SK_DPD_DEF
0,100002,1.0,-9461,0.1394,0.2630,0.0830,False,10.0,406597.5,24700.5,...,-10.000000,19.0,-1.0,-19.0,31.666667,1.0,0.0,24.0,0.0,0.0
1,100003,0.0,-16765,NaN,0.6220,0.3113,False,9.0,1293502.5,35698.5,...,-43.785714,28.0,-18.0,-77.0,607.137566,3.0,0.0,12.0,0.0,0.0
2,100004,0.0,-19046,0.7295,0.5560,NaN,False,11.0,135000.0,6750.0,...,-25.500000,4.0,-24.0,-27.0,1.666667,1.0,0.0,4.0,0.0,0.0
3,100006,0.0,-19005,NaN,0.6504,NaN,False,5.0,312682.5,29686.5,...,-9.619048,21.0,-1.0,-20.0,36.947619,3.0,0.0,5.0,0.0,0.0
4,100007,0.0,-19932,NaN,0.3228,NaN,False,5.0,513000.0,21865.5,...,-33.636364,66.0,-1.0,-77.0,510.265734,5.0,0.0,24.0,0.0,0.0


In [25]:
train_df.TARGET = train_df.TARGET.astype(int)
#train_df = train_df.replace(-np.inf, np.nan)
#train_df = train_df.replace(np.inf, np.nan)

train_df = label_encode_it(train_df)
train_df  = train_df.fillna(train_df.mean())
train_df = train_df[~train_df.isin([np.nan, np.inf, -np.inf]).any(1)]
for col in list(train_df):
    if train_df[col].isnull().any():
        train_df.drop(col, axis=1, inplace=True)
dataset = train_df.values
# split into input (X) and output (Y) variables
X = dataset[:,2:]
y = dataset[:,1]
y=y.astype('int')
print(X)


[] **********
[[-9461 0.1394 0.263 ... 24.0 0.0 0.0]
 [-19046 0.7295 0.556 ... 4.0 0.0 0.0]
 [-18850 0.5405 0.7144 ... 10.0 0.0 0.0]
 ...
 [-13210 0.3376 0.7495 ... 6.0 0.0 0.0]
 [-10010 0.297 0.635 ... 4.0 0.0 0.0]
 [-19070 0.1809 0.3716 ... 12.0 0.0 0.0]]


In [31]:
print(X.shape, y.shape)
#train_df[~train_df.isin([np.nan, np.inf, -np.inf]).any(1)]
# for col in list(train_df):
#     if train_df[col].isnull().any():
#         print(col)

(64, 544) (64,)


In [34]:
# baseline model
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=544, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy', auc])
    return model


In [35]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/100
50/50 [==============================] - 1s 15ms/step - loss: 0.6226 - acc: 0.8400 - auc: 0.6290
Epoch 2/100
50/50 [==============================] - 0s 941us/step - loss: 0.4790 - acc: 0.9600 - auc: 0.5084
Epoch 3/100
50/50 [==============================] - 0s 980us/step - loss: 0.3602 - acc: 0.9600 - auc: 0.6711
Epoch 4/100
50/50 [==============================] - 0s 1ms/step - loss: 0.2626 - acc: 0.9600 - auc: 0.7493
Epoch 5/100
50/50 [==============================] - 0s 818us/step - loss: 0.1817 - acc: 0.9600 - auc: 0.7250
Epoch 6/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1339 - acc: 0.9600 - auc: 0.7542
Epoch 7/100
50/50 [==============================] - 0s 860us/step - loss: 0.1021 - acc: 0.9600 - auc: 0.7869
Epoch 8/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0814 - acc: 0.9600 - auc: 0.8141
Epoch 9/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0661 - acc: 0.9800 - auc: 0.8432
Epoch 10/100
50/50 

50/50 [==============================] - 0s 1ms/step - loss: 6.8735e-04 - acc: 1.0000 - auc: 0.9978
Epoch 76/100
50/50 [==============================] - 0s 1ms/step - loss: 6.6946e-04 - acc: 1.0000 - auc: 0.9979
Epoch 77/100
50/50 [==============================] - 0s 862us/step - loss: 6.4903e-04 - acc: 1.0000 - auc: 0.9979
Epoch 78/100
50/50 [==============================] - 0s 894us/step - loss: 6.3382e-04 - acc: 1.0000 - auc: 0.9980
Epoch 79/100
50/50 [==============================] - 0s 934us/step - loss: 6.1607e-04 - acc: 1.0000 - auc: 0.9980
Epoch 80/100
50/50 [==============================] - 0s 926us/step - loss: 6.0309e-04 - acc: 1.0000 - auc: 0.9981
Epoch 81/100
50/50 [==============================] - 0s 1ms/step - loss: 5.8732e-04 - acc: 1.0000 - auc: 0.9981
Epoch 82/100
50/50 [==============================] - 0s 807us/step - loss: 5.7019e-04 - acc: 1.0000 - auc: 0.9982
Epoch 83/100
50/50 [==============================] - 0s 956us/step - loss: 5.5671e-04 - acc: 1.000

/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/100
52/52 [==============================] - 1s 16ms/step - loss: 0.6285 - acc: 0.7692 - auc: 0.5734
Epoch 2/100
52/52 [==============================] - 0s 1ms/step - loss: 0.4782 - acc: 0.9615 - auc: 0.7944
Epoch 3/100
52/52 [==============================] - 0s 936us/step - loss: 0.3626 - acc: 0.9423 - auc: 0.8398
Epoch 4/100
52/52 [==============================] - 0s 1ms/step - loss: 0.2637 - acc: 0.9423 - auc: 0.8478
Epoch 5/100
52/52 [==============================] - 0s 1ms/step - loss: 0.1934 - acc: 0.9423 - auc: 0.8756
Epoch 6/100
52/52 [==============================] - 0s 1ms/step - loss: 0.1445 - acc: 0.9423 - auc: 0.8858
Epoch 7/100
52/52 [==============================] - 0s 1ms/step - loss: 0.1129 - acc: 0.9615 - auc: 0.8990
Epoch 8/100
52/52 [==============================] - 0s 1ms/step - loss: 0.0925 - acc: 0.9808 - auc: 0.9166
Epoch 9/100
52/52 [==============================] - 0s 1ms/step - loss: 0.0784 - acc: 0.9808 - auc: 0.9252
Epoch 10/100
52/52 [=====

...

Epoch 1/100
51/51 [==============================] - 1s 20ms/step - loss: 0.6208 - acc: 0.8627 - auc: 0.8406
Epoch 2/100
51/51 [==============================] - 0s 789us/step - loss: 0.4473 - acc: 0.9412 - auc: 0.4567
Epoch 3/100
51/51 [==============================] - 0s 773us/step - loss: 0.3201 - acc: 0.9608 - auc: 0.6216
Epoch 4/100
51/51 [==============================] - 0s 767us/step - loss: 0.2252 - acc: 0.9608 - auc: 0.6914
Epoch 5/100
51/51 [==============================] - 0s 799us/step - loss: 0.1650 - acc: 0.9608 - auc: 0.7290
Epoch 6/100
51/51 [==============================] - 0s 873us/step - loss: 0.1233 - acc: 0.9608 - auc: 0.7312
Epoch 7/100
51/51 [==============================] - 0s 810us/step - loss: 0.0963 - acc: 0.9608 - auc: 0.7777
Epoch 8/100
51/51 [==============================] - 0s 825us/step - loss: 0.0793 - acc: 0.9608 - auc: 0.8142
Epoch 9/100
51/51 [==============================] - 0s 751us/step - loss: 0.0677 - acc: 0.9608 - auc: 0.8370
Epoch 10/10

51/51 [==============================] - 0s 938us/step - loss: 8.2632e-04 - acc: 1.0000 - auc: 0.9974
Epoch 75/100
51/51 [==============================] - 0s 828us/step - loss: 7.9526e-04 - acc: 1.0000 - auc: 0.9974
Epoch 76/100
51/51 [==============================] - 0s 933us/step - loss: 7.6936e-04 - acc: 1.0000 - auc: 0.9975
Epoch 77/100
51/51 [==============================] - 0s 764us/step - loss: 7.4690e-04 - acc: 1.0000 - auc: 0.9976
Epoch 78/100
51/51 [==============================] - 0s 874us/step - loss: 7.2567e-04 - acc: 1.0000 - auc: 0.9976
Epoch 79/100
51/51 [==============================] - 0s 756us/step - loss: 7.0802e-04 - acc: 1.0000 - auc: 0.9977
Epoch 80/100
51/51 [==============================] - 0s 819us/step - loss: 6.8671e-04 - acc: 1.0000 - auc: 0.9978
Epoch 81/100
51/51 [==============================] - 0s 813us/step - loss: 6.6959e-04 - acc: 1.0000 - auc: 0.9978
Epoch 82/100
51/51 [==============================] - 0s 775us/step - loss: 6.5314e-04 - acc:

/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/100
52/52 [==============================] - 1s 24ms/step - loss: 0.7562 - acc: 0.2885 - auc: 0.4168
Epoch 2/100
52/52 [==============================] - 0s 1ms/step - loss: 0.6107 - acc: 0.8846 - auc: 0.4163
Epoch 3/100
52/52 [==============================] - 0s 971us/step - loss: 0.5013 - acc: 0.9423 - auc: 0.5552
Epoch 4/100
52/52 [==============================] - 0s 882us/step - loss: 0.4025 - acc: 0.9423 - auc: 0.6619
Epoch 5/100
52/52 [==============================] - 0s 1ms/step - loss: 0.3131 - acc: 0.9423 - auc: 0.7080
Epoch 6/100
52/52 [==============================] - 0s 1ms/step - loss: 0.2425 - acc: 0.9615 - auc: 0.7555
Epoch 7/100
52/52 [==============================] - 0s 993us/step - loss: 0.1870 - acc: 0.9615 - auc: 0.7928
Epoch 8/100
52/52 [==============================] - 0s 878us/step - loss: 0.1462 - acc: 0.9615 - auc: 0.8201
Epoch 9/100
52/52 [==============================] - 0s 959us/step - loss: 0.1173 - acc: 0.9615 - auc: 0.8445
Epoch 10/100
52/5

52/52 [==============================] - 0s 927us/step - loss: 9.6395e-04 - acc: 1.0000 - auc: 0.9974
Epoch 76/100
52/52 [==============================] - 0s 919us/step - loss: 9.3560e-04 - acc: 1.0000 - auc: 0.9974
Epoch 77/100
52/52 [==============================] - 0s 1ms/step - loss: 9.0720e-04 - acc: 1.0000 - auc: 0.9975
Epoch 78/100
52/52 [==============================] - 0s 1ms/step - loss: 8.7910e-04 - acc: 1.0000 - auc: 0.9976
Epoch 79/100
52/52 [==============================] - 0s 891us/step - loss: 8.5711e-04 - acc: 1.0000 - auc: 0.9976
Epoch 80/100
52/52 [==============================] - 0s 1ms/step - loss: 8.3267e-04 - acc: 1.0000 - auc: 0.9977
Epoch 81/100
52/52 [==============================] - 0s 1ms/step - loss: 8.1129e-04 - acc: 1.0000 - auc: 0.9977
Epoch 82/100
52/52 [==============================] - 0s 935us/step - loss: 7.9171e-04 - acc: 1.0000 - auc: 0.9978
Epoch 83/100
52/52 [==============================] - 0s 900us/step - loss: 7.6960e-04 - acc: 1.0000 

In [ ]:
test_df = read_csv_data('shiv_test', debug=True)

In [43]:
estimators[1][1].predict_proba()

TypeError: predict_proba() missing 1 required positional argument: 'x'